# Set up and global variables

In [ ]:
import os

from pathlib import Path
from copy import deepcopy

import pandas as pd
import numpy as np
import json

from matplotlib.colors import ListedColormap
from IPython.display import display, HTML
from tqdm import tqdm

from src.prioritization import *

In [ ]:
os.environ["CONFIG_ENV"] = "debug"
if False:
    os.environ["CONFIG_ENV"] = "production"

from config import load_config
config = load_config()

DEBUG = config["DEBUG"]

# input data
TRAINING_DATA_PATH = config['PATHS']['development_set']
VALIDATION_DATA_PATH = config['PATHS']['evaluation_set']
# storage
STORAGE = config['PATHS']['storage']

***

# Loading data

In [ ]:
items = pd.read_csv(STORAGE / 'items.csv', index_col=0)
defects = pd.read_csv(STORAGE / f'defects.csv', index_col=0)

train_log = pd.read_csv(TRAINING_DATA_PATH / 'log.csv', index_col=0, parse_dates=['time'])
train_defect_log = pd.read_csv(TRAINING_DATA_PATH / 'defect_log.csv', index_col=0)
train_defect_log.columns = train_defect_log.columns.astype(int)

test_log = pd.read_csv(VALIDATION_DATA_PATH / 'log.csv', index_col=0, parse_dates=['time'])
test_defect_log = pd.read_csv(VALIDATION_DATA_PATH / 'defect_log.csv', index_col=0)
test_defect_log.columns = test_defect_log.columns.astype(int)

***

# Heuristics

In [ ]:
data = items, defects

models = [
    TaskCommonModel(*data),
    TaskCharacteristicModel(*data),
    StudentCommonModel(*data),
    StudentCharacteristicModel(*data),
    StudentEncounteredBeforeModel(*data),
    DefectMultiplicityModel(*data),
    SeverityModel(*data),
]

models = {model.get_model_name(): model for model in models}

In [ ]:
for name, model in (pbar :=tqdm(models.items(), desc="Training Models")):
    pbar.set_description(f"Training {name}")
    model.update(train_log, train_defect_log)

***

# Pilot testing

## function definitions

In [ ]:
# Written with ChatGPT and Gemini

def _generate_submission_html(sub_num: int, submission: pd.Series,
                                         defect_counts: pd.Series, # Pass original counts
                                         active_models: dict, items: pd.DataFrame, defects: pd.DataFrame,
                                         show_scores: bool, show_probabilities: bool, show_levels: bool) -> str:
    """Generate HTML for a submission, displaying basic information about the submission and the selected tables with model outputs."""
    task_id = submission['item']
    task_row = items.loc[task_id]
    present_mask = defect_counts > 0
    present_defects_index = present_mask[present_mask].index

    # --- Calculate necessary outputs ---
    raw_scores_df, prioritized_scores_df, discrete_levels_df, rank_series = None, None, None, None

    # Calculate raw scores if needed for display or sorting
    if show_scores or show_probabilities or show_levels:
        temp_raw_scores = {name: model._calculate_scores(submission, defect_counts).loc[present_defects_index]
                           for name, model in active_models.items()}
        raw_scores_df = pd.DataFrame(temp_raw_scores)
        if not raw_scores_df.empty:
            numeric_cols = raw_scores_df.select_dtypes(include=np.number).columns
            raw_scores_df['SUM_RANK'] = raw_scores_df[numeric_cols].rank(ascending=False, axis=0).sum(axis=1)
            rank_series = raw_scores_df['SUM_RANK']

    # Calculate prioritized scores if requested
    if show_probabilities and not present_defects_index.empty:
        temp_prio_scores = {name: model.prioritize(submission, defect_counts).loc[present_defects_index]
                            for name, model in active_models.items()}
        prioritized_scores_df = pd.DataFrame(temp_prio_scores)

    # Calculate discrete levels if requested
    if show_levels and not present_defects_index.empty:
        temp_disc_levels = {name: model.discretize(submission, defect_counts).loc[present_defects_index]
                            for name, model in active_models.items()}
        discrete_levels_df = pd.DataFrame(temp_disc_levels)

    # --- HTML Structure ---
    css = """
<style>
.tooltip { position: relative; display: inline-block; cursor: help; border-bottom: 1px dashed #fff; }
.tooltip .tooltiptext { visibility: hidden; width: 400px; background-color: #1e1e1e; color: #f5f5f5; text-align: left; border-radius: 6px; padding: 10px; position: absolute; z-index: 1000; top: 50%; left: 100%; transform: translateY(-50%) translateX(10px); opacity: 0; transition: opacity 0.3s; font-family: monospace; white-space: normal; box-shadow: 0 4px 8px rgba(0,0,0,0.5); }
.tooltip:hover .tooltiptext { visibility: visible; opacity: 1; }
.tooltiptext strong { color: #FFD700; } .tooltiptext pre { background-color: #333; padding: 5px; border-radius: 3px; overflow-x: auto; }
.panel { background-color: #333 !important; color: #f5f5f5 !important; border: 1px solid #555 !important; padding: 10px; border-radius: 5px; }
.hdr { background-color: #444 !important; color: #f5f5f5 !important; }
.cell { border: 1px solid #555; padding: 8px; text-align: center; background-color: #333; color: #f5f5f5; }
.cell-left { text-align: left; font-weight: bold; }
.level-cell { font-weight: bold; font-size: 0.9em; }
.score-cell { font-size: 0.85em; color: #bbb; }
</style>
    """
    html_output = css + f"""
    <div style="border: 2px solid #555; margin: 20px 0; padding: 15px; border-radius: 8px; background-color: #222; color: #f5f5f5;">
        <div style="background-color: #4CAF50; color: white; padding: 8px; border-radius: 5px; margin-bottom: 15px;">
            <h3 style="margin: 0;">SUB {sub_num + 1} | Task: {task_id} ({task_row['display name']})</h3>
            <p style="margin: 0; font-size: 0.9em;">At: {submission['time']}</p>
        </div>
        <div style="display: flex; gap: 15px; margin-bottom: 15px;">
             <div class="panel instructions-panel" style="flex: 1;"><strong>Instructions:</strong> <p style="font-size: 0.9em; max-height: 100px; overflow-y: auto;">{task_row['instructions']}</p></div>
             <div style="flex: 1.5; border: 1px solid #ddd; padding: 10px; border-radius: 5px; background-color: #2e2e1e; color: #f5f5f5; font-family: monospace;"><strong>Code:</strong> <pre style="margin: 0; max-height: 150px; overflow-y: auto; white-space: pre-wrap;">{submission['answer']}</pre></div>
        </div>
    """

    def _generate_table_html(scores_df, title, mode):
        """Generate the HTML table for the model output tables."""
        if scores_df is None or scores_df.empty: return f"<h4 style='margin-top: 20px;'>{title}</h4><p>No defects present or table not shown.</p>"

        lmap15 = { 0: ("Abs", "#404040"), 1: ("VL", "#d9f0a3"), 2: ("L", "#addd8e"), 3: ("M", "#fdae6b"), 4: ("H", "#f46d43"), 5: ("VH", "#d73027") }
        lmapZ = { 0: ("Avg", "#ffffbf"), 1: ("HF", "#fdae61"), 2: ("VHF", "#d7191c"), -1: ("LF", "#abd9e9"), -2: ("VLF", "#2c7bb6"), "abs": ("Abs", "#404040")}
        tc15 = { 0: "#ccc", 1: "#000", 2: "#000", 3: "#000", 4: "#fff", 5: "#fff" }
        tcZ = { 0: "#000", 1: "#000", 2: "#fff", -1: "#000", -2: "#fff", "abs": "#ccc" }

        tbl = f"<h4 style='margin-top: 20px;'>{title}</h4><table style='width:100%; border-collapse:collapse; font-size:0.9em;'><thead><tr class='hdr'><th class='cell cell-left' style='width:25%;'>Defect</th>"
        for name in active_models.keys(): tbl += f"<th class='cell' style='width:15%;'>{name}<br><span style='font-size:0.8em;'></span></th>"
        tbl += "</tr></thead><tbody>"

        idx_to_sort = rank_series.sort_values(ascending=True).index if rank_series is not None else scores_df.index

        for defect_id in idx_to_sort:
            if defect_id not in scores_df.index: continue
            dr = defects.loc[defect_id]

            # Tooltip
            tt_cont = f"<strong>T:</strong> {dr['defect type']}<br><strong>D:</strong> {dr['description']}<br>"
            def add_code(t, c): return f"<strong>{t}:</strong><pre>{str(c).replace('<','&lt;').replace('>','&gt;')}</pre>" if pd.notna(c) and c!="" else ""
            tt_cont += add_code("Ex", dr['code example'])
            tt_cont += add_code("Fix", dr['code fix example'])
            tbl += f"<tr><td class='cell data-cell-left'><div class='tooltip'>{dr['display name']}<span class='tooltiptext'>{tt_cont}</span></div></td>"

            # Cells
            for name, model in active_models.items():
                val = scores_df.loc[defect_id, name]
                cell_html = "<td class='cell data-cell'>" # Default

                if mode == 'discrete':
                    lvl = int(val); isZ = model.get_discretization_scale() == '-2-2'
                    cmap = lmapZ if isZ else lmap15; tmap = tcZ if isZ else tc15
                    key = lvl if isZ else lvl; key = "abs" if lvl==0 and isZ else key
                    txt, bg = cmap.get(key, ("?", "#888")); tc = tmap.get(key, "#000")
                    disp = f"{key if isZ and key!='abs' else lvl}({txt})"
                    cell_html = f"<td class='cell level-cell' style='background-color:{bg}; color:{tc};'>{disp}</td>"
                elif mode == 'prioritized' or mode == 'raw':
                    score = val
                    if mode == 'prioritized': min_s, max_s = 0.0, 1.0
                    else: s_col = scores_df[name]; min_s, max_s = s_col.min(), s_col.max()
                    norm = 0.5 if max_s == min_s or pd.isna(min_s) or pd.isna(max_s) else np.interp(score, [min_s, max_s], [0.1, 1.0])
                    r,g,b = 255, 255-int(230*norm), 255-int(230*norm)
                    color = f"rgb({max(0,r)},{max(0,g)},{max(0,b)})"; tc = "#fff" if norm > 0.6 else "#000"
                    cell_html = f"<td class='cell score-cell' style='background-color:{color}; color:{tc};'>{score:.3f}</td>"

                tbl += cell_html
            tbl += "</tr>"
        tbl += "</tbody></table>"
        return tbl

    # --- Conditionally Add Aditional Tables ---
    if present_defects_index.empty:
         html_output += "<h4 style='margin-top: 20px;'>No Defects Present</h4>"
    else:
        if show_scores: html_output += _generate_table_html(raw_scores_df.drop(columns=['SUM_RANK'], errors='ignore'), "Raw Scores (_calculate_scores)", 'raw')
        if show_probabilities: html_output += _generate_table_html(prioritized_scores_df, "Prioritized Weights (Softmax)", 'prioritized')
        if show_levels: html_output += _generate_table_html(discrete_levels_df, "Discrete Levels", 'discrete')

    html_output += "</div>" # Close main container div
    return html_output

def pilot_test_user(user_id: int, all_models: dict, log: pd.DataFrame, defect_log: pd.DataFrame, items: pd.DataFrame, defects: pd.DataFrame,
                    show_scores: bool = False, show_probabilities: bool = False, show_levels: bool = False):
    """Analyze a user's submission history, displaying selected views based on flags."""
    user_log = log[log['user'] == user_id].sort_values('time')
    user_defect_log = defect_log.loc[user_log.index]

    if user_log.empty: display(HTML(f"<h2>No submissions for User ID: {user_id}</h2>")); return

    active_models = {name: deepcopy(model) for name, model in all_models.items()}
    for sub_num, (index, submission) in enumerate(tqdm(user_log.iterrows(), total=len(user_log), desc=f"User {user_id}")):
        defect_counts = user_defect_log.loc[index]

        html_output = _generate_submission_html(
            sub_num, submission, defect_counts, active_models, items, defects,
            show_scores, show_probabilities, show_levels
        )
        display(HTML(html_output))

        for model in active_models.values(): model.update(submission, defect_counts)


## defect scores

In [ ]:
pilot_test_user(test_log.iloc[10]['user'], models, test_log, test_defect_log, items, defects, show_scores=True, show_levels=True)